<div align="center" style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding: 10px;">
    <a href="https://github.com/encord-team/text-to-image-eval/actions?query=workflow%3ATests" target="_blank" style="text-decoration:none"><img src="https://github.com/encord-team/text-to-image-eval/actions/workflows/tests.yml/badge.svg?branch=main"></a>
    <a href="https://www.python.org/downloads/release/python-3119/" target="_blank" style="text-decoration:none"><img src="https://img.shields.io/badge/python-3.11%2B-blue" alt="Python Versions"></a>
    <a target="_blank" style="text-decoration:none"><img alt="PRs Welcome" src="https://img.shields.io/badge/PRs-Welcome-blue"></a>
    <img alt="License" src="https://img.shields.io/github/license/encord-team/text-to-image-eval">
  </div>
  <div style="flex: 1; padding: 10px;">
    <a href="https://github.com/encord-team/encord-notebooks" target="_blank" style="text-decoration:none"><img alt="Encord Notebooks" src="https://img.shields.io/badge/Encord_Notebooks-blue?logo=github&label=&labelColor=181717"></a>
    <a href="https://join.slack.com/t/encordactive/shared_invite/zt-1hc2vqur9-Fzj1EEAHoqu91sZ0CX0A7Q" target="_blank" style="text-decoration:none"><img alt="Join us on Slack" src="https://img.shields.io/badge/Join_Our_Community-4A154B?label=&logo=slack&logoColor=white"></a>
    <a href="https://twitter.com/encord_team" target="_blank" style="text-decoration:none"><img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/encord_team?label=%40encord_team&amp;style=social"></a>
  </div>
</div>

<h1 align="center">
    <img src="https://storage.googleapis.com/docs-media.encord.com/static/img/text-to-image-eval/tti-eval-banner-short.png" alt="tti-eval logo banner">
</h1>

# 📝 tti-eval | 🤖 Bring Your Own Model From Hugging Face Quickstart

Welcome to the _Bring Your Own Model From Hugging Face Quickstart_ guide for the `tti-eval` tool.
This notebook covers how to bring a model from Hugging Face to `tti-eval`.

In this notebook, you will explore:
- How to find the required information of the target model on the Hugging Face website.
- Generating and registering the new model definition.
- ⭐ Start using your model right away.

## Overview

`tti-eval` is a repository for benchmarking text-to-image models **on your own data**!

With this library, you can take an embedding model intended for jointly embedding images and text (like [CLIP](https://huggingface.co/openai/clip-vit-large-patch14-336)) and compute metrics for how well such model performs on classifying your custom dataset.

🟡 Prior to continuing, ensure that Colab's runtime type is set to `T4 GPU` instead of `CPU`. You can find this option under 'Runtime > Change runtime type > Hardware accelerator'. Additionally, note that the notebook may restart the runtime the first time you run it after installing Python 3.11. This restart is necessary as switching Python versions in Colab requires a runtime restart.

> Learn more about 📝 `tti-eval`:
> - [🖥️ CLI Quickstart](https://colab.research.google.com/drive/1SsKre_9vQ9SAIQyPFQmbUBXRKzoZv8AH?usp=sharing)
> - [🐍 Python Quickstart](https://colab.research.google.com/drive/1DNn5uH6rss8I4BZG4-Y0rayRkRwYXwMu?usp=sharing)
> - [🗄️ Bring Your Own Dataset From Encord Quickstart](https://colab.research.google.com/drive/1qESU9zx1cBnK3vUDpOeuH6uZjt-f5Cb6?usp=sharing)
> - [GitHub repository](https://github.com/encord-team/text-to-image-eval)

## Set up the repo and dependencies

In [ ]:
#@title 1. Install and set up Python 3.11 {display-mode: "form"}
# Credits to JanEricNitschke in https://github.com/pnxenopoulos/awpy/issues/248#issuecomment-1575170156

#The code below installs 3.11 (assuming you now have 3.10) and restarts environment, so you can run your cells.
import sys #for version checker
import os #for restart routine

if '3.11' in sys.version:
  print('You already have 3.11, nothing to install')
elif '3.10' in sys.version:
  print("Python version is: ", sys.version)

  print("Printing content of /usr/local/lib/python* to see available versions")
  !ls /usr/local/lib/python*

  #install python 3.11 and dev utils
  #you may not need all the dev libraries, but I haven't tested which aren't necessary.
  !sudo apt-get update -y > /dev/null
  !sudo apt-get install python3.11 python3.11-dev python3.11-distutils libpython3.11-dev > /dev/null
  !sudo apt-get install python3.11-venv binfmt-support  > /dev/null #recommended in install logs of the command above

  #change alternatives
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1 > /dev/null
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2 > /dev/null

  # install pip
  !curl -sS https://bootstrap.pypa.io/get-pip.py | python3.11  > /dev/null

  #install colab's dependencies
  !python3 -m pip install ipython==7.9.0 traitlets==5.7.1 jupyter psutil matplotlib setuptools ipython_genutils ipykernel jupyter_console notebook prompt_toolkit httplib2 astor  > /dev/null

  #minor cleanup
  !sudo apt autoremove > /dev/null

  #link to the old google package
  !ln -s /usr/local/lib/python3.10/dist-packages/google /usr/local/lib/python3.11/dist-packages/google > /dev/null

  #this is just to verify if 3.11 folder was indeed created
  print("Printing content of /usr/local/lib/python3.11/")
  !ls /usr/local/lib/python3.11/

  #restart environment so you don't have to do it manually
  os.kill(os.getpid(), 9)
else:
  print("Your out of the box Python is not 3.10, so probably the script will not work, so pls feel free to edit the script to ignore then check and re-run: ", sys.version)

You already have 3.11, nothing to install


In [ ]:
#@title 2. Clone the repo
!git clone https://github.com/encord-team/text-to-image-eval.git

Cloning into 'text-to-image-eval'...
remote: Enumerating objects: 828, done.
remote: Counting objects: 100% (410/410), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 828 (delta 284), reused 243 (delta 179), pack-reused 418
Receiving objects: 100% (828/828), 2.01 MiB | 34.22 MiB/s, done.
Resolving deltas: 100% (439/439), done.


In [ ]:
#@title 3. Install repo dependencies
%cd /content/text-to-image-eval/
!pip install poetry
!poetry config virtualenvs.in-project true # Configure Poetry to create virtual environments in the project folder
!poetry add ipykernel # Resolve Colab specific missing dependency
!poetry install --no-ansi

sys.path.insert(0, '/content/text-to-image-eval/.venv/lib/python3.11/site-packages')

## Bring your model from Hugging Face

Within `tti-eval`, the `HFModel` model type facilitates loading models from Hugging Face.

To register a model from Hugging Face, you will require the full title of the model (`<user>/<model-title>`). With this information, store the model definition as a JSON file in the `sources/models/` folder at the root of the repository, adhering to this structure:

```json
{
  "model_type": "HFModel",
  "title": "<title-in-tti-eval>",
  "title_in_source": "<full-model-title-in-hugging-face>"
}
```

### Find the full title of the model

Open the [Hugging Face website](https://huggingface.co/) and navigate to the target model. The model title is located below the search bar in the top-left corner. Please refer to the image for visual guidance.

![model-title-location-in-hugging-face](https://storage.googleapis.com/docs-media.encord.com/static/img/text-to-image-eval/model-title-location-hugging-face.png)

In [ ]:
#@markdown Overwrite the `title_in_source` variable with the actual model title from Hugging Face
title_in_source = "yainage90/fashion-pattern-clip"

### 2. Register the model to `tti-eval`

 With the full title of the model at hand, let's include the model definition as a JSON file in the `sources/models/` folder at the root of the repository, adhering to this structure:

```json
{
  "model_type": "HFModel",
  "title": "<title-in-tti-eval>",
  "title_in_source": "<full-model-title-in-hugging-face>"
}
```

In [ ]:
import json
from pathlib import Path

model_definition = {
  "model_type": "HFModel",
  "title": "finetuned-fashion",
  "title_in_source": title_in_source,
}

Path('./sources/models/finetuned-fashion.json').write_text(json.dumps(model_definition, indent=2), encoding="utf-8")

Your model has now been permanently incorporated into `tti-eval` and is ready to be utilized for any future tasks.

### [EXTRA] Test the model availability

In [ ]:
#@title Simple test to check that the model is available in the CLI
!poetry run tti-eval list --all

Available datasets are: Alzheimer-MRI, chest-xray-classification, LungCancer4Types, plants, rsicd, skin-cancer, sports-classification
Available models are: apple, bioclip, clip, eva-clip, fashion, finetuned-fashion, plip, pubmed, rsicd, rsicd-encord, siglip_large, siglip_small, street, tinyclip, vit-b-32-laion2b


In [ ]:
#@title Let's use Python to get the model
from tti_eval.model import ModelProvider

my_model = ModelProvider.get_model("finetuned-fashion")
my_model.title_in_source

/content/text-to-image-eval/.venv/lib/python3.11/site-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

'yainage90/fashion-pattern-clip'

In [ ]:
#@title Advanced test to build the embeddings of a sample dataset
#@markdown It's commented as it can take some time and is slower on a CPU configuration.

!poetry run tti-eval build --model-dataset finetuned-fashion/Alzheimer-MRI

Generating train split: 100% 5120/5120 [00:00<00:00, 40043.10 examples/s]
Generating test split: 100% 1280/1280 [00:00<00:00, 47302.65 examples/s]
Embedding dataset with finetuned-fashion: 100% 88/88 [00:45<00:00,  1.92it/s]
Embeddings saved successfully to file at `/content/text-to-image-eval/.cache/embeddings/Alzheimer-MRI/finetuned-fashion_train.npz`
Embedding dataset with finetuned-fashion: 100% 16/16 [00:07<00:00,  2.13it/s]
Embeddings saved successfully to file at `/content/text-to-image-eval/.cache/embeddings/Alzheimer-MRI/finetuned-fashion_validation.npz`


## Wrap Up

This notebook has provided an introduction to bringing a model from Hugging Face to `tti-eval`, showcasing how to find the full title of the relevant model, register the model definition in `tti-eval`, and start using it right away.

---

🟣 `tti-eval` is an open-source repository for benchmarking text-to-image models **on your own data**!
**Check out the project on [GitHub](https://github.com/encord-team/text-to-image-eval) and leave a star 🌟** if you like it.
Contributions are welcome!
Please feel free to open an issue or submit a pull request with your suggestions, bug fixes, or new features.

---

👉 Check out the 📖 [Encord Blog](https://encord.com/blog/) and 📺 [YouTube](https://www.youtube.com/@encord) channel to stay up-to-date with the latest in computer vision, foundation models, active learning, and data-centric AI.


## Next: Learn the how to bring your own dataset from Encord to `tti-eval`.

What's next? 👀 Dive into how to bring a dataset from Encord to `tti-eval`. In the Colab notebook, you will explore:
- How to find the required information of the dataset on the Encord website.
- Generating and registering the new dataset definition.
- ⭐ Start using your dataset right away.

[NEXT NOTEBOOK](https://colab.research.google.com/drive/1qESU9zx1cBnK3vUDpOeuH6uZjt-f5Cb6?usp=sharing) ➡️